<center style="font-size:48px;">Master Notebook</center>

# Libraries

In [ ]:
# Data Science
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
import geopandas as gpd

# Importing the Data

In [ ]:
cars = pd.read_csv('../Data/car-assignments.csv')
cc = pd.read_csv('../Data/cc_data.csv', encoding='cp1252', parse_dates=['timestamp'])
gps = pd.read_csv('../Data/gps.csv', parse_dates=['Timestamp'])
loyalty = pd.read_csv('../Data/loyalty_data.csv', encoding='cp1252', parse_dates=['timestamp'])